In [ ]:
# 该文件对 神经网络模型进行membership infernece attack，目前采用了4个攻击方法


In [15]:
# 导包
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import math
import sys
import urllib
import pickle
import argparse

sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
from ppsplit.attacks.membership_inference.Mentr_attack import MentrAttack # 包含了4种方法的攻击类
from target_model.models.splitnn_utils import split_weights_client

In [13]:
# 设置一些超参数
batch_size = 100
target_model_path = '/home/dengruijun/data/project/Inverse_efficacy/trained_models/Purchase100/model_best.pth'
split_layer = 3

In [11]:
# purchase 数据集 和 模型 导入
from target_model.models.PurchaseNet import PurchaseClassifier1

class_num = 100 # Purchase的分类类别数目

# 模型加载并切割：
model = PurchaseClassifier1(layer=split_layer)
pweights  = torch.load(target_model_path)['state_dict']
if split_layer < 8: # 
    pweights = split_weights_client(pweights ,model.state_dict())
model.load_state_dict(pweights)
model.eval()

# model = PurchaseClassifier1()
# model = torch.nn.DataParallel(model).cuda()
# checkpoint = torch.load(target_model_path)
# model.load_state_dict(checkpoint['state_dict'])
# model.eval()

# 加载数据集
shadow_train_loader, shadow_test_loader, target_train_loader, target_test_loader = prepare_purchase_data(batch_size=args.batch_size)


NameError: name 'split_weights_client' is not defined

In [ ]:
# Mentro attack 对象实例化
MIA = MentrAttack(num_classes = class_num)

# 攻击初始化，推理模型
# 这里 shadow model 和 target model 用的是相同的一个
MIA.prepare_model_performance(model, shadow_train_loader, shadow_test_loader, 
                              model, target_train_loader, target_test_loader)


In [ ]:
# 实施MIA攻击
# 设定要执行的攻击类型：默认是4种都执行
benchmark_methods = ['correctness','confidence','entropy','modified entropy'] # 可以取子集
MIA.mem_inf_benchmarks(all_methods=True)


In [ ]:
# 实施MIA攻击，只执行指定的2种攻击
benchmark_methods = ['correctness','modified entropy'] # 可以取子集
MIA.mem_inf_benchmarks(all_methods=False, benchmark_methods = benchmark_methods)